In [1]:
import os
import cv2
import numpy as np
import random
import sys
import testGMM

In [4]:
def trainGMM(K, img, max_iter):
    # user defined converge threshold
    tau = 0.00000000000000001

    def initialize():
        random_w = random.randint(0,img.shape[0])
        random_h = random.randint(0,img.shape[1])
        mean = [[img[random_w][random_h][0]],[img[random_w][random_h][1]],[img[random_w][random_h][2]]]
        # generate a random positive-semidefinete matrix as covariance matrix
        A = np.random.randint(1,50,(3,3))
        cov = np.dot(A, A.transpose())
        scaling = random.random() * 10.0
        return [scaling,mean,cov]

    params = [initialize() for cluster in range(K)]
    # Structure of para:
    # [[scale,mean,covariance],[scale,mean,covariance],[scale,mean,covariance]...]
    # scale is a int. Mean is a 3x1 matrix. Covariance is a 3x3 matrix

    # total_mean is the sum of all mean from different clusters
    total_mean = np.asmatrix([[-9999],[-9999],[-9999]])
    prev_total_mean = np.asmatrix([[9999],[9999],[9999]])
    iter = 0
    while iter <= max_iter and np.linalg.norm(total_mean - prev_total_mean) > tau:
        # update prev total mean
        prev_total_mean = total_mean

        # Expectation step - assign points to clusters, get cluster weight
        weights = []
        for cluster in range(K):
            # weight for a single cluster
            cluster_weights = np.zeros((img.shape[0], img.shape[1]))
            # cumulated weights add up all weights on a given pixel -- serving as denominator
            cumulated_weights = np.zeros((img.shape[0], img.shape[1]))
            cluster_scaling, cluster_mean, cluster_cov = params[cluster]

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    likelihood = testGMM.get_likelihood(pix, cluster_mean, cluster_cov)

                    ## calculate weight at position (w, h)
                    weight = cluster_scaling * likelihood
                    cumulated_weights[w][h] += weight
                    cluster_weights[w][h] = weight # probability of each pixel belonging to this cluster

            weights.append(cluster_weights) # weights for all clusters 1 to K,
            #weights[i][w][h]is the probability of the (w,h) pixel belonging to the ith cluster
        for cluster in range(K):
            weights[cluster] = np.divide(weights[cluster], cumulated_weights)

        # Maximization step - get new scaling, mean, and cov for each cluster
        for cluster in range(K):
            mean_sum = np.zeros((3,1)) # sums all weight*pixel RGB value on image
            cov_sum = np.zeros((3,3))
            sum_weights = sum(weights[cluster]) # sum of all the weights given a cluster
            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate mean
                    mean_sum += np.multiply(weights[cluster][w][h],pix)

            new_mean = np.divide(mean_sum, sum_weights)

            for w in range(len(img[:, 0, 0])):
                for h in range(len(img[0, :, 0])):
                    pix = np.asmatrix([[img[w][h][0]], [img[w][h][1]], [img[w][h][2]]])
                    # calculate covariance
                    cov_sum += np.multiply(weights[cluster][w][h], (pix - new_mean))@((pix - new_mean).T)
            new_cov = np.divide(cov_sum, sum_weights)
            new_scaling = sum_weights / (img.shape[0]*img.shape[1])
            mean_sum += mean_sum
            # update model
            params[cluster] = (new_scaling, new_mean, new_cov)
        iter += 1





In [5]:
input_dir = "train_images"
for img_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, img_name))
    trainGMM(5,img,200)
    break

ZeroDivisionError: float division by zero

In [13]:
A = [[4],[3]]
B = [[1],[0]]
np.linalg.norm(A)

5.0